https://dacon.io/competitions/official/235930/codeshare/6002?page=1&dtype=recent

In [1]:
!pip install -q annoy
!pip install -q FRUFS
!pip install -q pacmap

## Import

In [2]:
import random
import pandas as pd
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [65]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Optimizer, AdamW
from torch.optim.lr_scheduler import LambdaLR, CyclicLR, OneCycleLR
#from transformers.optimization import get_cosine_with_hard_restarts_schedule_with_warmup
#from transformers import get_cosine_with_hard_restarts_schedule_with_warmup

from sklearn.metrics import f1_score

# 모델 학습을 위해 CUDA 환경 설정. : 지피유 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'mps')

In [ ]:
def MacroF1Score(y_true, y_pred):
    f1_score(y_true, y_pred, average='macro')

In [67]:
EPOCHS = 50
LR = 1e-2
BS = 16384 # 2의 제곱승꼴
SEED = 41

In [68]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED) # Seed 고정

## Data Pre-processing

In [3]:
import os
os.chdir('/Users/lhs/Desktop/Machine_Sound_Data')

In [54]:
train = pd.read_csv('./train_.csv').drop(columns=['Unnamed: 0']) # 모두 정상 Sample
# train_df = train_df.iloc[:,2:]
test = pd.read_csv('./test_.csv').drop(columns=['Unnamed: 0'])
# test_df = test_df.iloc[:,2:]

In [57]:
'''
def get_mfcc_feature(df):
    features = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)
        
    return features

train_features = get_mfcc_feature(train_df)
test_features = get_mfcc_feature(test_df)
'''

"\ndef get_mfcc_feature(df):\n    features = []\n    for path in tqdm(df['SAMPLE_PATH']):\n        # librosa패키지를 사용하여 wav 파일 load\n        y, sr = librosa.load(path, sr=CFG['SR'])\n        \n        # librosa패키지를 사용하여 mfcc 추출\n        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])\n\n        y_feature = []\n        # 추출된 MFCC들의 평균을 Feature로 사용\n        for e in mfcc:\n            y_feature.append(np.mean(e))\n        features.append(y_feature)\n        \n    return features\n\ntrain_features = get_mfcc_feature(train_df)\ntest_features = get_mfcc_feature(test_df)\n"

In [77]:
X_train, x_val, Y_train, y_val = train_test_split(train, train['LABEL'], test_size=0.2)

In [79]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['LABEL'].values
            self.df = self.df.drop(columns=['LABEL']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            x = torch.from_numpy(self.df[index]).type(torch.FloatTensor)
            y = torch.FloatTensor([self.labels[index]])
            return x, y
            #self.x = self.df[index]
            #self.y = self.labels[index]
            #return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [83]:
train_dataset = MyDataset(df = X_train, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=6)

val_dataset = MyDataset(df = x_val, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=6)

In [85]:
# 계층 정규화
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

        self.init_weights()

    def init_weights(self):
        self.weight.data.fill_(1.0)
        self.bias.data.zero_()

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon) # 계층정규화 완료
        return self.weight * x + self.bias # wx+b
        
# 활성화 함수
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
        
class AutoEncoder1(nn.Module):
    def __init__(self):
        super(AutoEncoder1, self).__init__()
        
        
        self.ln = LayerNorm(5000)
        self.ln1 = LayerNorm(3500)
        self.ln2 = LayerNorm(2000)
        self.ln3 = LayerNorm(1000)
        
        self.upblock1 = nn.Sequential(nn.Linear(30, 1000), nn.BatchNorm1d(1000),GELU())
        self.upblock2 = nn.Sequential(nn.Linear(1000,2000), nn.BatchNorm1d(2000),GELU())
        self.upblock3 = nn.Sequential(nn.Linear(2000,3500), nn.BatchNorm1d(3500),GELU())
        self.upblock4 = nn.Sequential(nn.Linear(3500,5000), nn.BatchNorm1d(5000),GELU())

        self.downblock1 = nn.Sequential(nn.Linear(5000, 3500),nn.BatchNorm1d(3500),GELU())
        self.downblock2 = nn.Sequential(nn.Linear(3500, 2000),nn.BatchNorm1d(2000),GELU())
        self.downblock3 = nn.Sequential(nn.Linear(2000, 1000),nn.BatchNorm1d(1000),GELU())
        self.downblock4 = nn.Sequential(nn.Linear(1000, 300),nn.BatchNorm1d(300),GELU())
        
        self.fclayer = nn.Sequential(nn.Linear(300,30))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        upblock1_out = self.upblock1(x) 
        upblock2_out = self.upblock2(upblock1_out)
        upblock3_out = self.upblock3(upblock2_out)
        upblock4_out = self.upblock4(upblock3_out)
        
        downblock1_out = self.downblock1(self.ln(upblock4_out)) 
        skipblock1 = downblock1_out + upblock3_out
        downblock2_out = self.downblock2(self.ln1(skipblock1))
        skipblock2 = downblock2_out + upblock2_out
        downblock3_out = self.downblock3(self.ln2(skipblock2))
        skipblock3 = downblock3_out + upblock1_out 
        downblock4_out = self.downblock4(self.ln3(skipblock3))
        
        x = self.fclayer(downblock4_out)
         
        return x # 4

In [86]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        # Loss Function
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self):
        self.model.to(self.device)
        best_score = 0
        avg = 1
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                x = x.float().to(self.device)
                self.optimizer.zero_grad()

                _x = self.model(x)
                loss = self.criterion(x, _x)

                loss.backward()
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 0.95)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score <= score and avg > np.mean(train_loss):
                best_score = score
                avg = np.mean(train_loss)
                torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
                print('---------------------------')
                print(f'Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')
    
    def validation(self, eval_model, thr):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)

                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff)<thr, 1, 0).tolist()
                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

In [87]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [88]:
model = nn.DataParallel(AutoEncoder1())
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, 
                                                       threshold_mode='abs', min_lr=1e-8, verbose=True)
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

AssertionError: Torch not compiled with CUDA enabled

# Hyperparameter Setting

In [58]:
CFG = {
    'SR':16000,
    'N_MFCC':128, # MFCC 벡터를 추출할 개수 (<=128)
    'SEED':41
}

## Model Fit

In [14]:
import pacmap

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [137]:
from sklearn.metrics import accuracy_score

model.fit(X_train, y_train)
pred = model.predict(X_val)
accuracy_score(y_val, pred)

1.0

In [138]:
model.fit(train_mfcc.iloc[:,:-1], train_mfcc['LABEL'])
xgb_pred = model.predict(test_mfcc)

In [139]:
xgb_pred

array([0, 0, 1, ..., 0, 0, 1])

## Prediction

In [12]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량) 이므로 (0:정상, 1:불량)로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [13]:
test_pred = model.predict(test_features) # model prediction
# test_pred = get_pred_label(test_pred)

In [18]:
test_pred

array([ 1,  1,  1, ...,  1,  1, -1])

## Submission

In [95]:
submit = pd.read_csv('./sample_submission.csv')

In [140]:
submit['LABEL'] = xgb_pred
submit.head()

,SAMPLE_ID,LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,0


In [141]:
submit.LABEL.value_counts()

1    890
0    624
Name: LABEL, dtype: int64

In [98]:
import datetime

In [142]:
path = '/Users/lhs/Desktop/GitHub/Dacon/230116_Machine_Error_Sound/result/'

now = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
submit.to_csv(f'{path}{now}.csv',encoding='utf-8', index=False)